## Vegetation Prediction using Sentinel-2

In this computing lab, you will create a simple machine learning model to predict vegetation percentage from Sentinel-2 imagery. The Copernicus Sentinel-2 mission is organized as a constellation of identical satellites in the same orbit: Sentinel-2A launched in 2015, Sentinel-2B launched in 2017, and Sentinel-2C launched in September of this year. Each satellite carries a multispectral imager with 13 spectral bands to measure our land and vegetation.

<img src=images/sentinel-2.jpg alt="Sentinel 2" width=500>

### 1. Learning Objectives
The goals of this computing lab are to:

1. Familiarize yourself with the structure of Sentinel-2 data
2. Visualize sample Sentinel-2 imagery and corresponding vegetation labels
3. Create a supervised machine learning pipeline using Pytorch Lightning and develop e.g.
    - dataloaders
    - data augmentations & transformations
    - (one or more) machine learning models
    - validation metrics

### 2. Data Exploration

Let's start by exploring the satellite data you will work with. You can download Sentinel-2 data directly from ```Copernicus``` (https://browser.dataspace.copernicus.eu/) or via platforms like ```Google Earth Engine``` (https://earthengine.google.com/). After creating an account, these platforms allow you to visualize and query EO data via their web interfaces, and provide support to download larger amounts of data via the command line or python scripts.
After downloading, it is usually a non-trivial amount of work to prepare EO data for machine learning purposes. Depending on your application, you might want to mask clouds, calculate weekly/monthly/seasonal summaries, adjust the range of pixel values etc. In addition, since EO images are often very large, you need to decide on a patching strategy to create machine learning ready data tiles that are large enough to contain relevant information, but small enough to fit into memory. 

##### 2.1. Area of Interest

Luckily, we have already downloaded and prepared ML-ready data for you! For this lab, we will focus on data from 2020 in Oxfordshire. To download the data, we create a wkt shape file of the county boundaries of Oxfordshire, and used ```geetiles``` (https://github.com/rramosp/geetiles) to download spatially aligned data tiles. We highly recommend looking into this tool, if you plan to use ```Google Earth Engine``` for your research. If you run the cell below, you can visualize what the spatial coverage of the tiles looks like. Each tile covers 1280 m x 1280 m

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

# Step 1: Load the GeoJSON file
geojson_file = "oxfordshire_partitions_aschips_2c696f57ca1c0.geojson"
gdf = gpd.read_file(geojson_file)

# Step 2: Plot the GeoDataFrame with no fill color and thin outlines
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.5)

# Step 3: Add a basemap
ctx.add_basemap(ax, crs=gdf.crs.to_string())

# Step 4: Customize the plot
ax.set_title("Overview of available data tiles in Oxfordshire")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

plt.show()

**Question**: How many data tiles are there in total?

##### 2.2. Data Variables & Visualization

Now that we have looked at our area of interest, let's investigate the data we will use, saved in **PATH/TO/DATA**. In this folder, you find Sentinel-2 imagery (sentinel2-2020) and vegetation labels (vegetation-2020). The vegetation labels are provided by NASA, and were derived from MODIS measurements (see https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD44B).

In the data repository, each folder contains the same number of files, labeled with a unique identifier for each data tile shown above. In the cells below, pick a few identifiers and investigate what the data looks like.

In [ ]:
import os
# Define data paths
base_path = 'oxfordshire_partitions_aschips_2c696f57ca1c0' # TODO: Remove data path
s2_path = os.path.join(base_path, 'sentinel2-2020')
veg_path = os.path.join(base_path, 'vegetation-2020')

# Pick a sample tile to investigate
tile_id = '0b18813cdd183' # TODO: Remove tile_id

In [ ]:
import xarray as xr
import rioxarray
import rasterio

# Define sentinel-2 file path
s2_filename = os.path.join(s2_path, f'{tile_id}.tif')

# Open sentinel-2 image using xarray and rasterio
s2_ds = xr.open_dataset(s2_filename, engine='rasterio')
s2_ds

In [ ]:
# Define vegetation file path
veg_filename = os.path.join(veg_path, f'{tile_id}.tif')

# Open vegetation data using xarray and rasterio
veg_ds = xr.open_dataset(veg_filename, engine="rasterio")
veg_ds

**Question**: What can you learn about the data and variables stored?

As you can see, the vegetation labels contain information on the percent tree cover for the years 2016 - 2020 (although we will only use data from 2020 today).
Our Sentinel-2 data contains seasonal summaries (Spring: Mar - May, Summer: Jun - Aug, Fall: Sep - Nov, Winter: Dec - Feb) for the high resolution (10-20 m) channels:

<img src=images/sentinel-2-bands.png alt="Sentinel 2 Bands" width=400>

**Question**: Which bands do you estimate are the most useful for vegetation prediction?

**Question**: Plot a few example tiles, and investigate what the data looks like. Can you create a proper RGB image of the Sentinel-2 data? What do you learn about the resolution of the datasets?

In [ ]:
import numpy as np
# Select data variables
vars = ['summer_B4', 'summer_B3', 'summer_B2']

# Open dataset
ds = xr.open_dataarray(s2_filename, engine="rasterio")

# Select data variables
idx = [list(ds.long_name).index(v) for v in vars]
ds = ds.isel(band=idx)
# Convert data to numpy array
arr = ds.compute().values
# Normalize data between 0 and 1
arr = arr / np.nanmax(arr) 
# Change axis order for plotting
arr = np.moveaxis(arr, 0, -1)

plt.imshow(arr)
plt.title("Sentinel-2 RGB Image");

In [ ]:
# Select data variables
vars = ['Percent_Tree_Cover_2020']

# Open dataset
ds = xr.open_dataarray(veg_filename, engine="rasterio")

# Select data variables
idx = [list(ds.long_name).index(v) for v in vars]
ds = ds.isel(band=idx)
# Convert data to numpy array
arr = ds.compute().values
# Change axis order for plotting
arr = np.moveaxis(arr, 0, -1)

plt.imshow(arr)
plt.colorbar()
plt.title('Percent Tree Cover 2020');

### 3. Pipeline Development

Now that we have investigated our data, we will develop our training pipeline, consisting of a dataloader, model, trainer, and evaluation metrics. Note, since the resolution of the vegetation labels is much coarser than the resolution of the Sentinel-2 imaergy, we will focus on predicting the **mean vegetation percentage** of each tile, i.e. a regression task.

##### 3.1. Datasets & Dataloader

Let's start with creating our dataloader. For this, we will first create a dataset that loads one image at a time, and feed this to a Pytorch dataloader that will create batches for training. We have started developing the code for you below, but a few lines are missing that you should fill in. Make sure that you test your dataloader for different experimental configurations.

To make dataloading easier, we prepared a csv file for you (```oxfordshire_partitions_aschips_2c696f57ca1c0_splits.csv```), that contains information on which tiles should be used for training, validation, and testing. While there are different ways to split your data, we have opted to split our data by geographics bands (also using ```geetiles```). If you run the cell below, you can visualize what the data splits look like.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from matplotlib.patches import Patch

# Load the GeoJSON file
geojson_file = "oxfordshire_partitions_aschips_2c696f57ca1c0.geojson"
gdf = gpd.read_file(geojson_file)

fig, ax = plt.subplots(figsize=(10, 10))

# Plot each split separately
gdf[gdf['split'] == 'train'].plot(ax=ax, facecolor='red', edgecolor='black', linewidth=0.5)
gdf[gdf['split'] == 'val'].plot(ax=ax, facecolor='blue', edgecolor='black', linewidth=0.5)
gdf[gdf['split'] == 'test'].plot(ax=ax, facecolor='green', edgecolor='black', linewidth=0.5)

# Add a basemap
ctx.add_basemap(ax, crs=gdf.crs.to_string())

# Add a legend
# Create custom legend
legend_elements = [
    Patch(facecolor='red', edgecolor='black', label='Train'),
    Patch(facecolor='blue', edgecolor='black', label='Validation'),
    Patch(facecolor='green', edgecolor='black', label='Test')
]
ax.legend(handles=legend_elements, loc='upper right')

# Customize the plot
ax.set_title("Tiles & splits")
plt.xlabel("Longitude")
plt.ylabel("Latitude");

**Question**: What are some of the considerations when deciding on data splits?

In [ ]:
from torch.utils.data import Dataset
from typing import List

class Dataset(Dataset):
    def __init__(
        self, 
        data_dir: str,
        files: List[str],
        seasons: List[str],
        bands: List[str],
        ):

        super().__init__()

        self.data_dir = data_dir
        self.files = files
        self.seasons = seasons
        self.bands = bands

        self.s2_path = os.path.join(data_dir, 'sentinel2-2020')
        self.veg_path = os.path.join(data_dir, 'vegetation-2020')

        self.s2_bands = [f"{season}_{band}" for season in seasons for band in bands]
        self.veg_bands = ['Percent_Tree_Cover_2020']

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        # Compile file path
        s2_path = os.path.join(self.s2_path, f'{self.files[idx]}.tif')
        veg_path = os.path.join(self.veg_path, f'{self.files[idx]}.tif')

        # Open sentinel-2 image using xarray and rasterio
        s2_ds = xr.open_dataarray(s2_path, engine="rasterio")
        s2_ds = s2_ds.isel(band=[list(s2_ds.long_name).index(b) for b in self.s2_bands])
        s2_arr = s2_ds.compute().values

        # Normalize sentinel-2 data
        # Data is in image format with pixel values between [0, 255]
        s2_arr = s2_arr / 255.0

        # Replace Nan values with 0
        s2_arr = np.nan_to_num(s2_arr, nan=0.0)

        # Open vegetation data using xarray and rasterio
        veg_ds = xr.open_dataarray(veg_path, engine="rasterio")
        veg_ds = veg_ds.isel(band=[list(veg_ds.long_name).index(b) for b in self.veg_bands])
        veg_arr = veg_ds.compute().values
        mean_veg = (np.nanmean(veg_arr) / 100).astype(np.float32)

        return s2_arr, mean_veg

In [ ]:
import pandas as pd
from lightning.pytorch import LightningDataModule
from torch.utils.data import DataLoader
from typing import List, Callable

class Dataloader(LightningDataModule):
    def __init__(
        self,
        data_dir: str,
        split_file: str,
        seasons: List[str]=['winter', 'spring', 'summer', 'fall'],
        bands: List[str]=['B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'], # Red-Edge, NIR, SWIR and visible bands
        batch_size: int = 4,
    ):
        super().__init__()
        self.data_dir = data_dir
        self.split_file = pd.read_csv(split_file)
        self.seasons = seasons
        self.bands = bands

        self.batch_size = batch_size

        # Get list of files for each split
        self.train_files = self.split_file[self.split_file['split'] == 'train']['identifier'].tolist()
        self.val_files = self.split_file[self.split_file['split'] == 'val']['identifier'].tolist()
        self.test_files = self.split_file[self.split_file['split'] == 'test']['identifier'].tolist()

        self.train_dataset = Dataset(
            data_dir=self.data_dir,
            files=self.train_files,
            seasons=self.seasons,
            bands=self.bands
        )
        self.val_dataset = Dataset(
            data_dir=self.data_dir,
            files=self.val_files,
            seasons=self.seasons,
            bands=self.bands
        )
        self.test_dataset = Dataset(
            data_dir=self.data_dir,
            files=self.test_files,
            seasons=self.seasons,
            bands=self.bands
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset, 
            batch_size=self.batch_size, 
            shuffle=True,
            )
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset, 
            batch_size=self.batch_size, 
            shuffle=False,
            )
    
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset, 
            batch_size=self.batch_size, 
            shuffle=False,
            )

##### 3.2. Model

With a working dataloader, we can now turn to developing our machine learning model. For now, we will experiment with a simple CNN regression model. If you finish early, you are very welcome to experiment with adapting this model or implementing other approaches.

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from lightning.pytorch import LightningModule

class CNN_Regression(LightningModule):
    def __init__(
        self, 
        input_shape: tuple,
        output_dim: int = 1, # Regression output is just one value
        learning_rate: float = 1.5e-4,
    ):
        super().__init__()

        self.input_shape = input_shape
        self.input_channels = input_shape[0]
        self.output_dim = output_dim

        # define & log hyperparameters
        self.learning_rate = learning_rate
        self.save_hyperparameters()

        # model architecture
        self.conv1 = nn.Conv2d(
            in_channels=self.input_channels,
            out_channels=32,
            kernel_size=3,
            stride=1,
        )
        self.conv2 = nn.Conv2d(32, 32, 3, stride=2)
        self.conv3 = nn.Conv2d(32, 64, 3, stride=2)
        self.conv4 = nn.Conv2d(64, 64, 3, stride=2)

        # activation function
        self.activation_fn = F.relu
        self.output_activation_fn = F.elu

        # pooling layer
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2)

        # compute the flattened size of the output tensor
        hidden_dim = self._get_output_shape(self.input_shape)

        # linear layers for regression head
        self.fc1 = nn.Linear(hidden_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.output_layer = nn.Linear(128, output_dim)

        # criterion
        self.criterion = nn.MSELoss()

    def _get_output_shape(self, shape):
        """returns the size of the output tensor after the conv layers"""
        batch_size = 1
        # initialize random input tensor
        input = torch.rand(batch_size, *shape, requires_grad=True)
        # forward pass through the conv layers
        output_feature = self.feature_extractor(input)
        hidden_dim = output_feature.data.view(batch_size, -1).size(1)
        return hidden_dim
    
    def _get_number_parameters(self):
        """returns the number of trainable parameters"""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    # computations
    def feature_extractor(self, x):
        """extract features from the conv blocks"""
        x = self.activation_fn(self.conv1(x))
        x = self.pool1(self.activation_fn(self.conv2(x)))
        x = self.activation_fn(self.conv3(x))
        x = self.pool2(self.activation_fn(self.conv4(x)))
        return x

    def forward(self, x):
        """produce final model output"""
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1) # reshapes the tensor to (batch_size, hidden_dim)
        x = self.activation_fn(self.fc1(x))
        x = self.activation_fn(self.fc2(x))
        x = self.activation_fn(self.fc3(x))
        x = self.output_activation_fn(self.output_layer(x))
        return x
    
    def compute_loss(self, batch):
        input, labels = batch
        # forward pass
        predictions = self(input).squeeze()
        # calculate loss
        loss = self.criterion(predictions, labels)
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.compute_loss(batch)
        # log training loss
        self.log(
            "train_loss", loss, on_step=True, on_epoch=True, logger=True, prog_bar=True
        )
        return loss
    
    def validation_step(self, batch, batch_idx):
        loss = self.compute_loss(batch)
        # log validation loss
        self.log(
            "val_loss", loss, on_step=True, on_epoch=True, logger=True, prog_bar=True
        )
        return loss

    def test_step(self, batch, batch_idx):
        loss = self.compute_loss(batch)
        # log test loss
        self.log(
            "test_loss", loss, on_step=True, on_epoch=True, logger=True, prog_bar=True
        )
        return loss

##### 3.3. Training

Let's start the training. For this, we can use the Pytorch Lightning Trainer, which wraps all code for the training loop conveniently for us.

In [ ]:
from lightning.pytorch import seed_everything

seed_everything(42);

**Question**: Why do we need to define a random seed?

In [ ]:
# Initialize dataloader
dl = Dataloader(
    data_dir=base_path,
    split_file="oxfordshire_partitions_aschips_2c696f57ca1c0_splits.csv",
    batch_size=4,
    seasons=['winter', 'spring', 'summer', 'fall'],
    bands=['B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'],
)

In [ ]:
# Initialize model 
model = CNN_Regression(
    input_shape=(36, 128, 128),
    output_dim=1,
    learning_rate=1.5e-4
)

In [ ]:
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import TQDMProgressBar
from lightning.pytorch.loggers import TensorBoardLogger

logger = TensorBoardLogger("logs", name="vegetation-prediction")

num_epochs = 15

# Initialize the trainer
trainer = Trainer(
    max_epochs=num_epochs,
    callbacks=[TQDMProgressBar(refresh_rate=10)],
    logger=logger,
    devices=1,
    accelerator='auto',
)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

trainer.fit(model, dl)

##### 3.4. Metrics & Visualization

You can visualize the training and validation losses, for instance through tensorboard. If you run the next line, an interactive window should appear, that allows you to plot the loss curves.

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=logs/

Now that you have reached the end of the notebook, there are a few areas you can explore:

- How well does the model perform? Can you improve the prediction results? For instance, you might experiment with the learning rate, model architecture, loss functions, data augmentations etc.

- How does the availability of input channels impact the prediction results? Are all seasons required to predict yearly vegetation maps? How does the model perform if you give it more or fewer spectral channels? Does this match what you would expect?

- Are there any other other model architectures (or baselines) to implement?

- What other metrics could you log during training?

- Can you create prediction maps of your model (and the ground truth)? 